In [1]:
import os
HOME = os.getcwd()

# !git clone https://github.com/IDEA-Research/GroundingDINO.git

# %cd {HOME}/GroundingDINO
# %pip install -q -e .

# set model configuration file path
CONFIG_PATH = os.path.join(HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")

# %cd {HOME}/GroundingDINO/weights
# !wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth

# set model weight file path
WEIGHTS_NAME = "groundingdino_swint_ogc.pth"
WEIGHTS_PATH = os.path.join(HOME, "GroundingDINO/weights", WEIGHTS_NAME)

In [2]:
import torch
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
from groundingdino.util.inference import Model

grounding_dino_model = Model(model_config_path=CONFIG_PATH, model_checkpoint_path=WEIGHTS_PATH)

c:\Users\Hyosk5\miniconda3\envs\torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'CONFIG_PATH' is not defined

In [6]:
!git clone https://github.com/CASIA-IVA-Lab/FastSAM.git


Cloning into 'FastSAM'...


In [8]:
# install FastSAM
# !wget https://huggingface.co/spaces/An-691/FastSAM/resolve/main/weights/FastSAM.pt
# !wget -P FastSAM/weights https://huggingface.co/spaces/An-619/FastSAM/resolve/main/weights/FastSAM.pt

--2023-12-07 21:36:26--  https://huggingface.co/spaces/An-619/FastSAM/resolve/main/weights/FastSAM.pt
Resolving huggingface.co (huggingface.co)... 13.225.131.94, 13.225.131.93, 13.225.131.6, ...
Connecting to huggingface.co (huggingface.co)|13.225.131.94|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/71/c3/71c3ec5a83f3fc374665cec9240f614262e29088fca1a18732571e13c7084f1b/c0be4e7ddbe4c15333d15a859c676d053c486d0a746a3be6a7a9790d52a9b6d7?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27FastSAM.pt%3B+filename%3D%22FastSAM.pt%22%3B&Expires=1702211786&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwMjIxMTc4Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy83MS9jMy83MWMzZWM1YTgzZjNmYzM3NDY2NWNlYzkyNDBmNjE0MjYyZTI5MDg4ZmNhMWExODczMjU3MWUxM2M3MDg0ZjFiL2MwYmU0ZTdkZGJlNGMxNTMzM2QxNWE4NTljNjc2ZDA1M2M0ODZkMGE3NDZhM2JlNmE3YTk3OTBkNTJhOWI2ZDc%7EcmVzcG9uc2

In [1]:
import os
HOME = os.getcwd()

%cd {HOME}/FastSAM


d:\vision\FastSAM


c:\Users\Hyosk5\miniconda3\envs\torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
WARNING  Ultralytics settings reset to defaults. This is normal and may be due to a recent ultralytics package update, but may have overwritten previous settings. 
View and update settings with 'yolo settings' or at 'C:\Users\Hyosk5\AppData\Roaming\Ultralytics\settings.yaml'


In [12]:
from fastsam import FastSAM, FastSAMPrompt
import torch
import numpy as np
import cv2
import time
import clip

In [13]:
model = FastSAM('./weights/FastSAM.pt')

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(DEVICE)

cam = cv2.VideoCapture(0)

while cam.isOpened():

    suc, frame = cam.read()

    start = time.perf_counter()

    everything_results = model(
        source=frame
      , device=DEVICE
      , retina_masks=True
      , imgsz=1024
      , conf=0.4
      , iou=0.9
    )

    print(everything_results[0].masks.shape)
    print(everything_results[0].boxes.shape)
    print(everything_results[0].boxes[0].xyxy.cpu().numpy())

    for box in everything_results[0].boxes:
        box = box.xyxy.cpu().numpy()[0]
        print(box)
        cv2.rectangle(frame, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0, 255, 0), 2)

    prompt_process = FastSAMPrompt(frame, everything_results, device=DEVICE)

    # everything prompt
    ann = prompt_process.everything_prompt()

    print(ann)

    end = time.perf_counter()
    total_time = end - start
    fps = 1 / total_time

    img = prompt_process.plot_to_result(frame, annotations=ann)

    cv2.putText(frame, f"FPS: {int(fps)}", (20,70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)
    cv2.imshow('frame', frame)
    cv2.imshow('img', img)

    key = cv2.waitKey(20)
    if key == 27: # exit on ESC 
        break

cam.release()
cv2.destroyAllWindows()

cuda



0: 768x1024 12 objects, 35.1ms
Speed: 5.0ms preprocess, 35.1ms inference, 6.0ms postprocess per image at shape (1, 3, 1024, 1024)


torch.Size([12, 480, 640])
torch.Size([12, 6])
[[     526.36      389.55         640         480]]
[     526.36      389.55         640         480]
[          0           0         640         480]
[      484.6      121.45       514.6      142.49]
[     450.63      176.02      542.24         480]
[     175.88      79.105      210.37      115.96]
[      510.9      102.55         640       177.7]
[     187.76      83.489      209.68      107.17]
[     539.32           0         640      48.007]
[     565.85           0      587.48      26.691]
[     538.06       389.2         640      455.85]
[     88.722           0      258.01       56.38]
[     419.92           0      569.94      19.323]
tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 1., 1., 1.],
         [0., 0., 0.,  ..., 1., 1., 1.],
         [0., 0., 0.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 0., 0., 0.]

TypeError: FastSAMPrompt.plot_to_result() got multiple values for argument 'annotations'

In [3]:
import supervision as sv
print(sv.__ver)

0.6.0


In [7]:
from ultralytics import FastSAM
from ultralytics.models.fastsam import FastSAMPrompt
import torch
import numpy as np
import cv2
import time

model = FastSAM('FastSAM-x.pt')

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(DEVICE)

cam = cv2.VideoCapture(0)

while cam.isOpened():

    suc, frame = cam.read()

    start = time.perf_counter()

    everything_results = model(
        source=frame
      , device=DEVICE
      , retina_masks=True
      , imgsz=(640, 480)
      , conf=0.4
      , iou=0.9
    )

    # print(everything_results[0].masks.shape)
    # print(everything_results[0].boxes.shape)
    # print(everything_results[0].boxes[0].xyxy.cpu().numpy())

    for box in everything_results[0].boxes:
        box = box.xyxy.cpu().numpy()[0]
        # print(box)
        cv2.rectangle(frame, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0, 255, 0), 2)

    prompt_process = FastSAMPrompt(frame, everything_results, device=DEVICE)

    # everything prompt
    ann = prompt_process.everything_prompt()

    print(ann)

    end = time.perf_counter()
    total_time = end - start
    fps = 1 / total_time

    img = prompt_process.plot(frame, annotations=ann)

    cv2.putText(frame, f"FPS: {int(fps)}", (20,70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)
    cv2.imshow('frame', frame)
    cv2.imshow('img', img)

    key = cv2.waitKey(20)
    if key == 27: # exit on ESC 
        break

cam.release()
cv2.destroyAllWindows()

cuda

0: 384x480 11 objects, 17.6ms
Speed: 1.0ms preprocess, 17.6ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 480)
[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: ultralytics.engine.results.Masks object
names: {0: 'object'}
orig_img: array([[[  1,   4,   3],
        [  0,   3,   2],
        [  1,   4,   3],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]],

       [[  6,   6,   6],
        [  4,   3,   3],
        [  5,   3,   3],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]],

       [[  6,   8,  10],
        [  1,   3,   5],
        [  1,   3,   5],
        ...,
        [126, 133, 146],
        [128, 134, 147],
        [  0, 255,   0]],

       ...,

       [[  0,   2,   4],
        [  1,   3,   6],
        [  0,   1,   6],
        ...,
        [ 35,  34,  31],
        [ 35,  36,  31],
        [ 34,  35, 

TypeError: FastSAMPrompt.plot() got multiple values for argument 'annotations'